In [ ]:
import torch
import torchvision
import time
import numpy as np
from torchvision.models.detection import (
    fasterrcnn_resnet50_fpn,
    fcos_resnet50_fpn,
    retinanet_resnet50_fpn,
    ssd300_vgg16,
    ssdlite320_mobilenet_v3_large
)

def measure_inference_time(model, input_size=(3, 320, 320), num_iterations=100, warmup=10):
    """
    测量模型的推理时延
    
    Args:
        model: 目标检测模型
        input_size: 输入图像尺寸
        num_iterations: 测试迭代次数
        warmup: 预热迭代次数
    
    Returns:
        float: 平均推理时间(ms)
    """
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model = model.to(device)
    model.eval()
    
    # 创建随机输入数据 - 注意是3D张量的列表
    dummy_input = [torch.randn(*input_size).to(device)]
    
    # 预热
    print("Warming up...")
    with torch.no_grad():
        for _ in range(warmup):
            _ = model(dummy_input)
    
    # 测量推理时间
    times = []
    print("Testing inference time...")
    with torch.no_grad():
        for i in range(num_iterations):
            start_time = time.perf_counter()
            _ = model(dummy_input)
            torch.cuda.synchronize()
            end_time = time.perf_counter()
            times.append((end_time - start_time) * 1000)  # 转换为毫秒
    
    mean_time = np.mean(times)
    std_time = np.std(times)
    return mean_time, std_time

def main():
    # 初始化所有要测试的模型
    models = {
        'Faster R-CNN': fasterrcnn_resnet50_fpn(pretrained=False, pretrained_backbone=False),
        'FCOS': fcos_resnet50_fpn(pretrained=False, pretrained_backbone=False),
        'RetinaNet': retinanet_resnet50_fpn(pretrained=False, pretrained_backbone=False),
        'SSD300': ssd300_vgg16(pretrained=False, pretrained_backbone=False),
        'SSDLite320': ssdlite320_mobilenet_v3_large(pretrained=False, pretrained_backbone=False)
    }
    
    # 测试每个模型的推理时间
    results = {}
    for name, model in models.items():
        print(f"\nTesting {name}...")
        mean_time, std_time = measure_inference_time(model)
        results[name] = (mean_time, std_time)
    
    # 打印结果
    print("\n=== Inference Time Results ===")
    print("Model Name | Mean Time (ms) | Std Dev (ms)")
    print("-" * 45)
    for name, (mean, std) in results.items():
        print(f"{name:<12} | {mean:>12.2f} | {std:>11.2f}")

if __name__ == "__main__":
    main()